---
title: "Topology Final Project"
bibliography: bibliography.bib
---



# TODO
- Ellaborate/explain each section in more detail
- Compute persistent image
- Convert persistent image to vectors
- Perform clustering of persistent features using K-means and/or Hierarchical clustering
- Look into PCA/PCR for potential clusterings??
- Get redshifts for each galaxy


# Welcome Lori!
Hi Lori (I assume no one else is going to be looking at the current version of my project)!

I was thinking of presenting my final project as a website because 1) I have a website already, and 2) I can display code and results in a really easy way. Additionally, Quarto (which is how I made the website) has an option to save the output to slides, so I might do that for the actual presentation. We'll see!

Starting below is a *rough* draft of what I have done so far.



# Project Overview
This project combines astronomy, persistent homology, and machine learning to investigate topological properties in gravitationally lensed galaxies.

## Astronomy
- What is a gravitationally lensed galaxy?
- Why do we care about them?

## Persistent Homology
- What is it?
- How is it helpful?
- How has it been used in astronomy before?
- What do I plan 

## Machine Learning
- What is machine learning?
- Why do we need/want to perform clustering?
- How do the clustering algorithms work and what are they?

# Data
- What data do I have? What does it look like?
- Where did I get the data from?
- Who funded the data?
- When was the data collected?
- Why was the data collected?
- How do I plan on using the data?

The code chunk below imports the necessary modules for working with this data.


In [ ]:
from astroquery.esa.euclid import Euclid
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, ImageNormalize, PercentileInterval, AsinhStretch, LogStretch
import gtda.plotting
import pandas as pd
from astropy.io import ascii
import glob
import numpy as np
import os
from gtda.images import Binarizer
from gtda.images import RadialFiltration
from gtda.homology import CubicalPersistence
from gtda.diagrams import Scaler
from gtda.diagrams import PersistenceImage

Once all the libraries are imported, we can extract the data. Each lensed galaxy was found via a machine learning classification model, thus there existed a distribution of objects with associated probabilities that they were lensed galaxies, based on the model. Currently, this project only incorporates objects that had a "grade" of "A" (i.e. were of the best quality).


In [ ]:
# get coordinates from list of targets (only select best images)
all_targets = ascii.read("D:/data/targets/q1_discovery_engine_lens_catalog.csv")
mask = (all_targets["subset"] == "discovery_engine") & (all_targets["grade"] == "A")
targets = all_targets[mask]

The target list only contains the celestial coordinates of the grade-A lensed galaxies. Luckily, we can download cutouts of the targets. Note, this code chunk is intentially set to not run as all of the cutouts have already been extracted. Note, we have ~250 targets.


In [ ]:
# | eval: false
# don't run unless need redownload images!!!
# download the images for each target
empty_images = 0
for i in range(len(targets)):
    r, d = targets["right_ascension"][i], targets["declination"][i]
    coord = SkyCoord(r, d, unit=u.deg, frame='icrs')
    radius = u.Quantity(0.5, u.deg)

    print(f"r: {r}")
    print(f"d: {d}")
    print(coord)
    print(radius)

    # search euclid's mosaic product catalog for targets
    job = Euclid.cone_search(coordinate=coord, radius=radius, table_name="sedm.mosaic_product", ra_column_name="ra", dec_column_name="dec", columns="*", async_job=True, verbose=True)
    cone_results = job.get_results()
    example_file = cone_results[cone_results['instrument_name'] == 'VIS'][0]
    print(f"cone_results: {cone_results}")

    # save results to output path
    file_path = example_file["file_path"] + "/" + example_file["file_name"]
    instrument = example_file["instrument_name"]
    obs_id = example_file["tile_index"]
    radius = 0.1 * u.arcmin
    output_folder = 'D:/data/'

    print(f"file_path: {file_path}")
    print(f"instrument: {instrument}")
    print(f"obs_id: {obs_id}")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_file=output_folder + f'cutouts_0.1arcmin/target_{r}_{d}.fits'
    saved_cutout_filepath = Euclid.get_cutout(file_path=file_path,
        instrument=instrument, id=obs_id, 
        coordinate=coord, radius=radius, output_file=output_file)

    print("Cutout saved at", output_file)

    # looking at the cutout we made
    try:
        hdul = fits.open(output_file)
        print(fits.info(output_file))
        image_data = hdul[0].data
        plt.imshow(image_data, interpolation='nearest', cmap='gray', origin='lower', norm=ImageNormalize(image_data, interval=PercentileInterval(99.5),
                            stretch=AsinhStretch()))
        plt.colorbar()
        plt.show()
    except:
        empty_images += 1

print(empty_images)

Once the cutouts are extracted, we can display them so get a sense of what the data actually looks like. The code chunk below shows an example of what a gravitationally lensed galaxy looks like from the Euclid space telescope.


In [ ]:
# Make sure all images are the same shapes (119, 119)
fits_files = glob.glob("D:/data/cutouts_0.1arcmin/*.fits")

for file in fits_files:
    try:
        hdul = fits.open(file)
        image_data = hdul[0].data
        header = hdul[0].header
        wcs = WCS(header)
        if image_data.shape == (120, 120):
            print(image_data.shape)
            image_data = np.delete(image_data, -1, axis=0)
            image_data = np.delete(image_data, -1, axis=1)
        fits.writeto(f"D:/data/cutouts_0.1arcmin_119x119/{file.split("\\")[-1]}", image_data, header, overwrite=True)
    except:
        print(f"EMPTY (deleting): {file}")
        os.remove(file)

In [ ]:
# check that everything in folder is same dimensions
fits_files_clean = glob.glob("D:/data/cutouts_0.1arcmin_119x119/*.fits")

shapes = []
for file in fits_files_clean:
    hdul = fits.open(file)
    data = hdul[0].data
    shapes.append(data.shape)

np.unique(shapes)

# Analysis
Once we have the images, the next step is to compute the persistent homology for each image. The code chunk below displays the example image using Python, a Giotto-tda heatmap mehtod, and calculates the cubical persistence of the example image.


In [ ]:
# load in the image
plt.imshow(data.reshape(119,119), interpolation='nearest', cmap='gray', origin='lower', norm=ImageNormalize(data, interval=PercentileInterval(99.5),
                            stretch=AsinhStretch()))
plt.colorbar()
plt.show()

image_data = data.reshape((1,119,119))  # reshape image

In [ ]:
# invert the image
from gtda.images import Inverter

inverter = Inverter(max_value=np.max(image_data))
image_data_inverted = inverter.fit_transform(image_data)

inverter.plot(image_data_inverted)

In [ ]:
# binarize the image
from gtda.images import Binarizer

binarizer = Binarizer(threshold=0.04)
image_data_binarized = binarizer.fit_transform(image_data_inverted)
binarizer.plot(image_data_binarized)

In [ ]:
# from binary image to filtration
from gtda.images import HeightFiltration

height_filtration = HeightFiltration()
image_data_filtration = height_filtration.fit_transform(image_data_binarized)

height_filtration.plot(image_data_filtration, colorscale="jet")

In [ ]:
# from filtration to persistence diagram
from gtda.homology import CubicalPersistence

cubical_persistence = CubicalPersistence(n_jobs=-1, )
image_data_cubical = cubical_persistence.fit_transform(data)

cubical_persistence.plot(image_data_cubical)

In [ ]:
# scale
from gtda.diagrams import Scaler

scaler = Scaler()
image_data_scaled = scaler.fit_transform(image_data_cubical)

scaler.plot(image_data_scaled)

In [ ]:
# from persistence diagram to persistence image
from gtda.diagrams import PersistenceImage
pim = PersistenceImage(n_jobs=-1)
image_data_pim = pim.fit_transform(image_data_scaled)
pim.plot(image_data_pim, homology_dimension_idx=0, colorscale='jet')

In [ ]:
# convert to vector for h-clustering
image_data_pim.flatten().shape

In [ ]:
# convert to pipeline for speed
from sklearn.pipeline import Pipeline
# print(image_data.shape)
steps = [
    # invert image for superlevel filtration
    ("cubical_persistence", CubicalPersistence(n_jobs=-1)),
    ("scaling", Scaler()),
    ("persistence_image", PersistenceImage(n_jobs=-1))
]

image_pipeline = Pipeline(steps)

In [ ]:
# run the pipeline
pipeline_results = []
for file in fits_files_clean[:10]:
    try:
        hdul = fits.open(file)
        data = hdul[0].data
        pipeline_results.append(image_pipeline.fit_transform(data.reshape(1,119,119)))
    except:
        print(f"error with index {fits_files_clean.index(file)}")


# pim.plot(test, homology_dimension_idx=0, colorscale="jet")

In [ ]:
print(len(pipeline_results))

for i in pipeline_results:
    image_data_pim = pim.fit_transform(i)
    pim.plot(image_data_pim).show()

In [ ]:
# run clustering algorithm

In [ ]:
from gtda.plotting import plot_heatmap
from gtda.homology import CubicalPersistence
from gtda.diagrams import PersistenceImage
import numpy as np
# # image_data

plt.imshow(image_data, cmap='inferno', origin='lower', norm=ImageNormalize(image_data, interval=PercentileInterval(99.5), stretch=AsinhStretch()))
plt.colorbar()
plt.xlabel("""X [pixels]

FIGURE 1: Example image of a gravitationally lensed galaxy used in 
this project. The 0.1 arcmin x 0.1 arcmin image was taken from the
Euclid Q1 Data Release and is centered on the galaxy with a 
characteristic lensing effect to the immediate top right.
""")
plt.ylabel("Y [pixels] ")
plt.title("Gravitationally Lensed Galaxy")
plt.show()

custom_scale = [
    [0.0, "blue"],   # Start color for low values
    [0.2, "orange"] # Midpoint
]

plot_heatmap(image_data, colorscale=custom_scale, origin="lower")

persistence = CubicalPersistence(n_jobs=-1, reduced_homology=False, infinity_values=np.inf)
fit = persistence.fit_transform(image_data)
persistence.plot(fit, 119)
fit.shape, fit

The code chunk below displays the 0 and 1 dimensional persistence diagrams for the example image. 


In [ ]:
h0 = persistence.transform(image_data)
diagramsh0 = np.delete(h0, 2, axis=2)

# bdmax = np.unique(bd0)[-2]
# infmask = bd0 != np.inf
# bdstd = np.std(bd0[infmask])
# print(bdstd)
# print(bd0.shape)
# birth, death = np.delete(bd0, 1, axis=2).flatten(), np.delete(bd0, 0, axis=2).flatten()
# plt.scatter(x=birth, y=death)

In [ ]:
from ripser import Rips
from persim import PersImage
from persim import PersistenceImager

rips = Rips(maxdim=1, coeff=2)
diagrams = [rips.fit_transform(data) for data in image_data]
diagrams_h1 = [rips.fit_transform(data)[1] for data in image_data]

pimgr = PersistenceImager(pixel_size=1)
pimgr.fit(bd0)
imgs = pimgr.transform(bd0)


# Tutorial


In [ ]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
import numpy as np
from gtda.plotting import plot_heatmap

im8_idx = np.flatnonzero(y == "8")[0]
img8 = X[im8_idx].reshape(28, 28)
plot_heatmap(img8)
img8.shape

In [ ]:
from gtda.images import Binarizer

# Pick out index of first 8 image
im8_idx = np.flatnonzero(y_train == "8")[0]
# Reshape to (n_samples, n_pixels_x, n_pixels_y) format
im8 = X_train[im8_idx][None, :, :]

print(im8.shape)

binarizer = Binarizer(threshold=0.4)
im8_binarized = binarizer.fit_transform(im8)

binarizer.plot(im8_binarized)

In [ ]:
from gtda.images import RadialFiltration

radial_filtration = RadialFiltration(center=np.array([20, 6]))
im8_filtration = radial_filtration.fit_transform(im8_binarized)

radial_filtration.plot(im8_filtration, colorscale="jet")

In [ ]:
from gtda.homology import CubicalPersistence

cubical_persistence = CubicalPersistence(n_jobs=-1)
im8_cubical = cubical_persistence.fit_transform(im8_filtration)

cubical_persistence.plot(im8_cubical)

In [ ]:
from gtda.diagrams import Scaler

scaler = Scaler()
im8_scaled = scaler.fit_transform(im8_cubical)

scaler.plot(im8_scaled)

# Next Steps

The next steps are to:

- compute the persistence image
- vectorize the result
- repeat for all targets
- cluster the vectorized targets using k-means and hierarchical clustering.

# References
blah blah [@Chen2015, @Sousbie2011]